# Querying Stage

This is a working notebook to write and test the code that is used in our Google Cloud function.

In this stage, the RAG pipeline extracts the most pertinent context based on a user’s query and forwards it, along with the query, to the LLM to generate a response. This procedure equips the LLM with current knowledge that wasn’t included in its original training data. This also reduces the likelihood of hallucinations, a problem for LLMs when they invent answers for data they were insufficiently trained with. The pivotal challenges in this phase revolve around the retrieval, coordination, and analysis across one or several knowledge bases.

In [1]:
# Suppress Pydantic warnings since it's based in llamaindex
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

## Hard-coded stuff in this cell that will be replaced in the cloud function
* OPEN AI Key will be an environment variable
* Weaviate IP address that we will work on finding programmatically

In [2]:
import os
from dotenv import load_dotenv
import weaviate
from weaviate import Client
from llama_index import VectorStoreIndex
from llama_index.storage import StorageContext
from llama_index.vector_stores import WeaviateVectorStore
from llama_index.vector_stores.types import ExactMatchFilter, MetadataFilters

# Load the .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variables
OPENAI_KEY = os.getenv("OPENAI_KEY")

# Set the OpenAI key as an Environment Variable (for when it's run on GCS)
os.environ["OPENAI_API_KEY"] = OPENAI_KEY

# Current Weaviate IP
WEAVIATE_IP_ADDRESS = "34.42.138.162"

/Users/iankelk/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_config.py:267: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


In [3]:
# Custom prompt to exclude out of context answers
from llama_index.prompts import PromptTemplate

# template = ("We have provided context information below. If the answer to a query is not contained in this context, "
# #             "please only reply that it is not in the context."
# #             "please only reply that it is not in the context. If your response will be financial in nature, "
#             "make the first character of the completion 1, and if it is not financial, make the first character 0."
#             "After this initial number, 0 or 1, please continue your response as instructed previously."
#             "\n---------------------\n"
#             "{context_str}"
#             "\n---------------------\n"
#             "Given this information, please answer the question: {query_str}\n"
# )

template = ("We have provided context information below. If the answer to a query is not contained in this context, "
            "please only reply with the first two characters of '0 ', and then explain that the context does not include "
            "the information. If the information IS included in the context, and your response will be financial "
            "in nature, make the two characters of the completion '1 ', and if it is not financial, make the first "
            "character '0 '. After this initial number, 0 or 1, please continue your response as instructed previously. "
            "Here are some examples: "
            "0 The location of this company is not provided in the given context. "
            "1 The company's profits were very high that year. "
            "0 The company is located in Grand Rapids, Michigan. "
            "\n---------------------\n"
            "{context_str}"
            "\n---------------------\n"
            "Given this information, please answer the question: {query_str}\n"
)

qa_template = PromptTemplate(template)

## Hard-coded stuff in this cell that will be replaced in the cloud function
* The websiteAddress will be from the query string of the https request
* The timestamp will be from the query string of the https request
* The query will be from the query string of the https request

In [12]:
# client setup
client = weaviate.Client(url="http://" + WEAVIATE_IP_ADDRESS + ":8080")

# construct vector store
vector_store = WeaviateVectorStore(weaviate_client=client, index_name="Pages", text_key="text")

# setting up the indexing strategy 
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# setup an index for the Vector Store
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)

# Create exact match filters for websiteAddress and timestamp
website_address_filter = ExactMatchFilter(key="websiteAddress", value="ai21.com")
timestamp_filter = ExactMatchFilter(key="timestamp", value="2023-10-06T18-11-24")

# Create a metadata filters instance with the above filters
metadata_filters = MetadataFilters(filters=[website_address_filter, timestamp_filter])

# Create a query engine with the custom prompt and filters
query_engine = index.as_query_engine(text_qa_template=qa_template,
                                     streaming=True,
                                     filters=metadata_filters)

# Execute the query
query_str = "How was AI21 Studio a game changer?"
#query_str = "Where is AI21 located?"
streaming_response = query_engine.query(query_str)

# Print the response as it arrives
# streaming_response.print_response_stream()

# async def astreamer(generator):
#     try:
#         for i in generator:
#             yield (i)
#             await asyncio.sleep(.1)
#     except asyncio.CancelledError as e:
#         print('cancelled')

def process_streaming_response(streaming_response):
    financial = False

    for i, text in enumerate(streaming_response.response_gen):
        if i == 0:  # Skip the initial null character
            continue
        elif i == 1:  # Check for the financial flag
            financial = text == "1"
        elif i == 2:  # Handle the first word (strip leading space)
            print(text.lstrip(), end="", flush=True)
        else:         # For subsequent words, print as is
            print(text, end="", flush=True)

    if financial:
        print(" Financial flag set!", flush=True)

process_streaming_response(streaming_response)

AI21 Studio was a game changer for Verb.ai by helping them create a revolutionary writing tool for authors, improving brainstorming and expression. Financial flag set!


In [5]:
def extract_document_urls(streaming_response):
    urls = []
    for node_with_score in streaming_response.source_nodes:
        relationships = node_with_score.node.relationships
        for related_node_info in relationships.values():
            if related_node_info.node_type == "4":  # Corresponds to ObjectType.DOCUMENT
                urls.append(related_node_info.node_id)
    return urls

extracted_urls = extract_document_urls(streaming_response)
print("The following websites were used as references:\n")
print(extracted_urls)

The following websites were used as references:

['https://www.ai21.com/blog/verb-ai-case-study', 'https://www.ai21.com/blog']


In [6]:
query_str = "Who is Kim Kardashian?"
streaming_response = query_engine.query(query_str)

# Print the response as it arrives
# streaming_response.print_response_stream()
process_streaming_response(streaming_response)

The context does not include information about who Kim Kardashian is.

In [7]:
extracted_urls = extract_document_urls(streaming_response)
print("The following websites were used as references:\n")
print(extracted_urls)

The following websites were used as references:

['https://www.ai21.com/blog/ubisoft-case-study', 'https://www.ai21.com/blog/retail-personalization-using-ai-will-transform-the-industry']
